In [1]:
# Ref: https://qiita.com/noca/items/00646efd9d4a7302f522
from ipywidgets import Textarea
import io
if 'open' in globals():
    del open
if 'input' in globals():
    del input
original_open = open
class custom_open():
    def __init__(self):
        self.text = ''
    def __call__(self, file, *args, **kwargs):
        if file == 0:
            return io.StringIO(self.text)
        return original_open(file, *args, **kwargs)
    def updater(self, change):
        self.text = change["new"]
class custom_input():
    def __init__(self):
        self.__sio = io.StringIO('')
        self.shell = get_ipython()
        if self.shell.events.callbacks['pre_run_cell'] != []:
            self.shell.events.callbacks['pre_run_cell'] = []
        self.shell.events.register('pre_run_cell', self.pre_run_cell)
    def __call__(self):
        return self.__sio.readline().strip()
    def pre_run_cell(self, info):
        text = self.shell.user_ns.get('text_area', None).value
        self.__sio = io.StringIO(text)
open = custom_open()
input = custom_input()
text_area = Textarea()
text_area.observe(open.updater, names='value')
display(text_area)

Textarea(value='')

In [106]:
H, W, N, h, w = map(int, input().split())
A = [list(map(int, input().split())) for _ in range(H)]

N_MAX = 301

## !!! 3次元配列で初期化するとTLEする
# cnt = [[[0]*(W+1) for __ in range(H+1)] for _ in range(N_MAX)]
cnt= [0] * (W+1) * (H+1) * N_MAX
tot = [0] * N_MAX

for i in range(H):
    for j in range(W):
        a = A[i][j]
        # cnt[a][i+1][j+1] = 1
        cnt[((j+1)*H+i+1)*N_MAX+a] = 1
        tot[a] += 1
        
# _sum = [[[0]*(W+1) for __ in range(H+1)] for _ in range(N_MAX)]
_sum = [0] * (W+1) * (H+1) * N_MAX

for a in range(1, N_MAX):
    if not tot[a]:
        continue
        
    for i in range(1, H+1):
        for j in range(1, W+1):
            # _sum[a][i][j] =  _sum[a][i][j-1] + cnt[a][i][j]
            _sum[(j*H+i)*N_MAX+a] = _sum[((j-1)*H+i)*N_MAX+a] + cnt[(j*H+i)*N_MAX+a]

    for j in range(1, W+1):
        for i in range(1, H+1):
            # _sum[a][i][j] +=  _sum[a][i-1][j]
            _sum[(j*H+i)*N_MAX+a] += _sum[(j*H+i-1)*N_MAX+a]

for k in range(H-h+1):
    ans_arr = []
    k += 1
    for l in range(W-w+1):
        l += 1
        # print(k,l)
        kk = k + h - 1
        ll = l + w - 1
        ans = 0
        for a in range(1, N_MAX):
            if not tot[a]:
                continue
            # num = _sum[a][kk][ll] - _sum[a][kk][l-1] - _sum[a][k-1][ll] + _sum[a][k-1][l-1]
            num = _sum[(ll*H+kk)*N_MAX+a] - _sum[((l-1)*H+kk)*N_MAX+a] - _sum[(ll*H+k-1)*N_MAX+a] + _sum[((l-1)*H+k-1)*N_MAX+a]
            # print(k, l, a, num, tot[a])
            if tot[a] - num != 0:
                ans += 1
        ans_arr.append(ans)
    print(*ans_arr)

4 4 3
5 3 4
